In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Download necessary packages

In [ ]:
!pip install kulc

In [ ]:
!git clone https://github.com/ahmadelsallab/MultiCheXNet.git

# Explore dataset

In [ ]:
import pandas as pd 
import cv2
import numpy as np
import os
from glob import glob
import math
import matplotlib.pyplot as plt

import re
import html
import string
import unicodedata
from nltk.tokenize import word_tokenize

In [ ]:
df  =pd.read_csv("/kaggle/input/chest-xrays-indiana-university/indiana_reports.csv")
df.head()

In [ ]:
dff = pd.read_csv('../input/chest-xrays-indiana-university/indiana_projections.csv')
dff.head()

In [ ]:
df.Problems.

In [ ]:
df['findings'].iloc[0:10].tolist()

In [ ]:
df.shape

In [ ]:
df['impression'].unique().shape

In [ ]:
df['MeSH'].unique().tolist()[:20]

In [ ]:
img = cv2.imread('/kaggle/input/chest-xrays-indiana-university/images/images_normalized/1_IM-0001-3001.dcm.png')

In [ ]:
plt.imshow(img)
plt.show()

In [ ]:
df2 = pd.read_csv("/kaggle/input/chest-xrays-indiana-university/indiana_projections.csv")
df2.head()

In [ ]:
df2.projection.unique()

# Build vocab

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow

**Text cleaner**

In [ ]:
def remove_special_chars(text):
    re1 = re.compile(r'  +')
    x1 = text.lower().replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>', 'u_n').replace(' @.@ ', '.').replace(
        ' @-@ ', '-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x1))


def remove_non_ascii(text):
    """Remove non-ASCII characters from list of tokenized words"""
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')


def to_lowercase(text):
    return text.lower()

def remove_punctuation(text):
    """Remove punctuation from list of tokenized words"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)


def replace_numbers(text):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    return re.sub(r'\d+', '', text)


def remove_whitespaces(text):
    return text.strip()


def remove_stopwords(words, stop_words):
    """
    :param words:
    :type words:
    :param stop_words: from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
    or
    from spacy.lang.en.stop_words import STOP_WORDS
    :type stop_words:
    :return:
    :rtype:
    """
    return [word for word in words if word not in stop_words]


def stem_words(words):
    """Stem words in text"""
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in words]

def lemmatize_words(words):
    """Lemmatize words in text"""

    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

def lemmatize_verbs(words):
    """Lemmatize verbs in text"""

    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word, pos='v') for word in words])

def text2words(text):
    return word_tokenize(text)

def normalize_text( text):
    text = remove_special_chars(text)
    text = remove_non_ascii(text)
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = replace_numbers(text)
    #words = text2words(text)
    #stop_words = stopwords.words('english')
    #words = remove_stopwords(words, stop_words)
    #words = stem_words(words)# Either stem ovocar lemmatize
    #words = lemmatize_words(words)
    #words = lemmatize_verbs(words)

    return text
  
def normalize_corpus(corpus):
    return [normalize_text(t) for t in corpus]
  

In [ ]:
df= df.dropna(subset=['findings'])

In [ ]:
df['findings_cleaned'] = df['findings'].swifter.apply(normalize_text)

In [ ]:
df['findings_cleaned'] = 'startseq '+df['findings_cleaned']+' endseq'

In [ ]:
num_words = []
for row in df['findings_cleaned'].tolist():
    num_words.append(len(word_tokenize(row)))

In [ ]:
num_words= np.array(num_words)
print("min length             : ", num_words.min())
print("max length             : ", num_words.max())
print("50th percentile length : ", np.percentile(num_words,50))
print("75th percentile length : ", np.percentile(num_words,75))
print("90th percentile length : ", np.percentile(num_words,90))
print("95th percentile length : ", np.percentile(num_words,95))
print("98th percentile length : ", np.percentile(num_words,98))
print("98th percentile length : ", np.percentile(num_words,99))


In [ ]:
vocab_size = 10000
max_len = 100

tok = Tokenizer(num_words=vocab_size,  oov_token='UNK' )
tok.fit_on_texts(df['findings_cleaned'].tolist())

vocab_size = len(tok.word_index) + 1
vocab_size

# Data loader from Repo

In [ ]:
from MultiCheXNet.data_loader.indiana_dataloader import get_train_validation_generator
from tensorflow.keras.applications.densenet import preprocess_input

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess_input

In [ ]:
max_vocab_size=10000
max_len=100

csv_path1  ="/kaggle/input/chest-xrays-indiana-university/indiana_reports.csv"
csv_path2  ="/kaggle/input/chest-xrays-indiana-university/indiana_projections.csv"
img_path   ="/kaggle/input/chest-xrays-indiana-university/images/images_normalized/"

train_dataloader, val_dataloader, vocab_size, tok = get_train_validation_generator(csv_path1,csv_path2,img_path, max_vocab_size,max_len,preprocess=vgg_preprocess_input)

In [ ]:
X,Y = next(enumerate(train_dataloader))[1]

In [ ]:
plt.imshow(X[0][0])
plt.show()

# Build Model

In [ ]:
from MultiCheXNet.evaluation.report_gen_evaluation import evaluate_from_dataloader

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam

In [ ]:
## Input layers
img_input = layers.Input(shape= (256,256,3)) 
report_input= layers.Input(shape= (max_len-1,))

## Encoder ######################

Densenet_model = tf.keras.applications.DenseNet121(
            include_top=False,
            #weights="imagenet",
            input_shape=(256,256,3),
        )
number_of_encoder_layers=  len(Densenet_model.layers)

encoder_output = Densenet_model(img_input)
encoder_output = layers.Flatten()(encoder_output)
encoder_output = layers.Dropout(0.2)(encoder_output)
encoder_output = layers.Dense(512,activation='relu')(encoder_output)

##decoder ########################

#layers
gru_layer =  layers.GRU(512,activation='relu', return_sequences=True)
dense_layer= layers.Dense(vocab_size,activation='softmax')
embedding_layer = layers.Embedding(vocab_size, 300, mask_zero=True)
dropout = layers.Dropout(0.2)

# decoder model
embedding_output = embedding_layer(report_input)
gru_output = gru_layer(embedding_output, initial_state=encoder_output )
gru_output = dropout(gru_output)
output  = dense_layer(gru_output)
model = Model([img_input,report_input ],output)

##Inference models ################

#encoder_inference model
encoder_model = Model(img_input,encoder_output)

#decoder_inference model
prev_hidden_state= layers.Input(shape= (512))
report_input2 = layers.Input(shape= (1,))
embedding_output2= embedding_layer(report_input2)
gru_output2 = gru_layer(embedding_output2, initial_state=prev_hidden_state )
output2 = dense_layer(gru_output2)
decoder_model = Model([report_input2,prev_hidden_state],[output2,gru_output2])

In [ ]:
model.summary()

In [ ]:
plot_model(model)

In [ ]:
epochs =5
lr=1e-3
model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(lr))

In [ ]:
hist = model.fit_generator( train_dataloader,
                    validation_data = val_dataloader,
                    epochs = epochs
                )

In [ ]:
y= hist.history['loss']
y_val = hist.history['val_loss']

x = range(0,len(y))
plt.plot(x,y)
plt.plot(x,y_val)
plt.show()

In [ ]:
model.save_weights("weights.hdf5")

In [ ]:
from MultiCheXNet.evaluation.report_gen_evaluation import get_predictions_from_data_loader
from copy import deepcopy

val_dataloader_tmp = deepcopy(val_dataloader)
val_dataloader_tmp.nb_iteration  = 3
GT , preds = get_predictions_from_data_loader(val_dataloader_tmp,tok,encoder_model, decoder_model,max_len,decoder_type='GRU')




In [ ]:
index=0
print(GT[index])
print(("====================================="))
print(preds[index])

In [ ]:
index=1
print(GT[index])
print(("====================================="))
print(preds[index])

In [ ]:
index=2
print(GT[index])
print(("====================================="))
print(preds[index])

In [ ]:
index=3
print(GT[index])
print(("====================================="))
print(preds[index])

In [ ]:
index=4 
print(GT[index])
print(("====================================="))
print(preds[index])

In [ ]:
evaluate_from_dataloader(val_dataloader,tok,encoder_model,decoder_model,max_len,decoder_type='GRU')

In [ ]:
#validation score: 0.6121


# (0.1625980645932062,
#  0.4032345032275961,
#  0.5798753018059997,
#  0.6350074828122863)

# # Model - Show and tell

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.backend as K

In [ ]:
## Input layers
img_input = layers.Input(shape= (256,256,3)) 
report_input= layers.Input(shape= (max_len-1,))

## Encoder ######################

Densenet_model = tf.keras.applications.DenseNet121(
            include_top=False,
            weights=None,#"imagenet",
            input_shape=(256,256,3),
        )
number_of_encoder_layers=  len(Densenet_model.layers)

encoder_output = Densenet_model(img_input)
encoder_output = layers.Flatten()(encoder_output)

X_img = layers.Dropout(0.5)(encoder_output)
X_img = layers.Dense(300, use_bias = False,activation='relu' ,
                        kernel_regularizer=regularizers.l2(1e-4),
                        name = 'dense_img')(X_img)
X_img = layers.BatchNormalization(name='batch_normalization_img')(X_img)
X_img = layers.Lambda(lambda x : K.expand_dims(x, axis=1))(X_img)

##decoder ########################

X_text = layers.Embedding(vocab_size, 300, mask_zero = True, name = 'emb_text')(report_input)
X_text = layers.Dropout(0.5)(X_text)

# Initial States


LSTMLayer = layers.LSTM(300, return_sequences = True, return_state = True, dropout=0.5, name = 'lstm')
output_layer = layers.TimeDistributed(layers.Dense(vocab_size, activation='softmax',
                                 kernel_regularizer = regularizers.l2(1e-4), 
                                 bias_regularizer = regularizers.l2(1e-4)), name = 'time_distributed_softmax')
# Take image embedding as the first input to LSTM
_, a, c = LSTMLayer(X_img)

A, _, _ = LSTMLayer(X_text, initial_state=[a, c])
output = output_layer(A)



model  = Model(inputs=[img_input, report_input], outputs=output, name='NIC_greedy_inference_v2')


##Inference models ################

#encoder_inference model
encoder_model = Model(img_input,[a,c])

# Decoder model ###################

a0 = layers.Input(shape=(300,))
c0 = layers.Input(shape=(300,))

A, alast, clast = LSTMLayer(X_text, initial_state=[a0, c0])
output = output_layer(A)



decoder_model = Model([report_input,a0,c0],[output,alast,clast])



In [ ]:
model.summary()

In [ ]:
plot_model(model)

In [ ]:
epochs =5
lr=1e-4
model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(lr))

In [ ]:
hist = model.fit_generator( train_dataloader,
                    validation_data = val_dataloader,
                    epochs = epochs
                )

In [ ]:
y= hist.history['loss']
y_val = hist.history['val_loss']

x = range(0,len(y))
plt.plot(x,y)
plt.plot(x,y_val)
plt.show()

In [ ]:
# model.save_weights("weights.hdf5")

In [ ]:
feat0_1, feat0_2 = encoder_model(np.expand_dims(X0[0],axis=0))
feat1_1, feat1_2 = encoder_model(np.expand_dims(X0[1],axis=0))
print(np.array(feat0_1==feat1_1).sum())
print(np.array(feat0_2==feat1_2).sum())


In [ ]:
from MultiCheXNet.evaluation.report_gen_evaluation import get_predictions_from_data_loader
from copy import deepcopy

val_dataloader_tmp = deepcopy(val_dataloader)
val_dataloader_tmp.nb_iteration  = 3
GT , preds = get_predictions_from_data_loader(val_dataloader_tmp,tok,encoder_model, decoder_model,max_len,decoder_type='LSTM')

In [ ]:
for index in range(8):
    print(GT[index])
    print(("====================================="))
    print(preds[index])
    print("")
    print("")

In [ ]:
evaluate_from_dataloader(val_dataloader,tok,encoder_model,decoder_model,max_len,decoder_type='LSTM')

In [ ]:
#val_loss: 0.7740



# (0.1420931304748732,
#  0.37695242468363727,
#  0.5568932125465332,
#  0.6139645141892464)



# Dr's Model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam


In [ ]:
img_input = layers.Input(shape= (256,256,3)) 
report_input= layers.Input(shape= (max_len-1,))

## Encoder ######################

Densenet_model = tf.keras.applications.DenseNet121(
            include_top=False,
            weights="imagenet",
            input_shape=(256,256,3),
        )

number_of_encoder_layers=  len(Densenet_model.layers)

encoder_output = Densenet_model(img_input)
encoder_output = layers.Flatten()(encoder_output)

fe1 = layers.Dropout(0.5)(encoder_output)
fe2 = layers.Dense(256, activation='relu')(fe1)

# sequence model
emb_layer =  layers.Embedding(vocab_size, 256, mask_zero=True)
dropout1_layer= layers.Dropout(0.5)
LSTM_layer=  layers.LSTM(256, return_sequences = True,return_state=True)

se1 = emb_layer(report_input)
se2 = dropout1_layer(se1)
se3,_,_ = LSTM_layer(se2)
D1  =layers.Dense(256 , activation='relu')
D2 = layers.Dense(vocab_size, activation='softmax')

# decoder model
decoder1 = layers.add([fe2, se3])
decoder2 = D1(decoder1)
outputs = D2(decoder2)

# tie it together [image, seq] [word]
model = Model(inputs=[img_input, report_input], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# summarize model
print(model.summary())
plot_model(model, to_file='model.png', show_shapes=True)


# Encoder model
encoder_model = Model(inputs=[img_input], outputs=fe2)

#decoder model
report_input1 = layers.Input(shape= (1,))
fe22 = layers.Input(shape=(256,))

se11 = emb_layer(report_input1)
se22 = dropout1_layer(se11)
se33,_,c33 = LSTM_layer(se22)

decoder11 = layers.add([fe22, se33])
decoder22 = D1(decoder11)
outputss = D2(decoder22)

decoder_model = Model(inputs=[fe22, report_input1 ], outputs=[outputss,se33,c33,fe22])

In [ ]:
epochs = 5
lr=1e-3
model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(lr))

In [ ]:
hist = model.fit_generator( train_dataloader,
                    validation_data = val_dataloader,
                    epochs = epochs
                )

In [ ]:
y= hist.history['loss']
y_val = hist.history['val_loss']

x = range(0,len(y))
plt.plot(x,y)
plt.plot(x,y_val)
plt.show()

In [ ]:
initializer = tf.keras.initializers.GlorotNormal()
a0 = initializer(shape=(1,256))
c0 = initializer(shape=(1,256))

In [ ]:
import numpy as np

def tokens_to_text(tokens,tok,end_token='endseq'):
    sentence=""
    for token in tokens:
        if token ==0:
            break
        
        word = tok.index_word[token]
        
        if word==end_token:
            break
            
        sentence+= word+" "
        
    sentence = sentence.strip()
    
    return sentence


def greedy_inference(input_img, tok,encoder_model, decoder_model,max_len,start_token="startseq",end_token='endseq',decoder_type="LSTM"):
    if decoder_type=='LSTM':
        a0,c0  =encoder_model(np.expand_dims(input_img,axis=0))
    elif decoder_type=='GRU': 
        hidden_layer  =encoder_model(np.expand_dims(input_img,axis=0))
        initializer = tf.keras.initializers.GlorotNormal()
        a0 = initializer(shape=(1,256))
        c0 = initializer(shape=(1,256))
        
    word = tok.word_index[start_token]
    
    words = []
    
    for index in range(max_len):
        if decoder_type=='LSTM':
            word_probs , a0,c0 = decoder_model.predict([[np.array([word]),a0,c0]])
        elif decoder_type=='GRU': 
            word_probs , hidden_layer = decoder_model.predict([[np.array([word]),hidden_layer]])
            hidden_layer=hidden_layer[0]
        
        word = np.argmax(word_probs)
        
        try:
            if tok.index_word[word]==end_token:
                break
        except:
            pass
        
        words.append(word)
        
    words = tokens_to_text(words,tok)
    return words

In [ ]:
def kk():
    return 1,2,3
l = kk()

In [ ]:
def ll(i , l1,l2,l3):
    print(i)
    print(l1)
    print(l2)
    print(l3)
    

In [ ]:
evaluate_from_dataloader(val_dataloader,
                         tok,
                         encoder_model,
                         decoder_model,
                         max_len,
                         decoder_type="GRU"

                        )

In [ ]:
#training without any pretrained weights, this model predicts all xxx

(0.003765473458262969,
 0.061363453767392925,
 0.18738976959694617,
 0.24771647859476956)

In [ ]:
from MultiCheXNet.evaluation.report_gen_evaluation import get_predictions_from_data_loader
from copy import deepcopy

In [ ]:
val_dataloader_tmp = deepcopy(val_dataloader)
val_dataloader_tmp.nb_iteration  = 3

In [ ]:
GT , preds = get_predictions_from_data_loader(val_dataloader_tmp,tok,encoder_model, decoder_model,max_len,decoder_type='GRU')

In [ ]:
index = 0
print(GT[index])
print("================================")
print(preds[index])


In [ ]:
index = 1
print(GT[index])
print("================================")
print(preds[index])


In [ ]:
index = 2
print(GT[index])
print("================================")
print(preds[index])


In [ ]:
index = 3
print(GT[index])
print("================================")
print(preds[index])


In [ ]:
index = 4
print(GT[index])
print("================================")
print(preds[index])


In [ ]:
index = 5
print(GT[index])
print("================================")
print(preds[index])


In [ ]:
decoder_model.summary()

In [ ]:
plot_model(decoder_model)

# Attention Model

In [ ]:
import keras.backend as K
from keras.layers import Layer, Dense, TimeDistributed, Concatenate, InputSpec,  RNN
from keras.layers.wrappers import Wrapper
import numpy as np
import tensorflow as tf

class ScaledDotProductAttention(Layer):
    """
        Implementation according to:
            "Attention is all you need" by A Vaswani, N Shazeer, N Parmar (2017)
    """

    def __init__(self, return_attention=False, **kwargs):    
        self._return_attention = return_attention
        self.supports_masking = True
        super(ScaledDotProductAttention, self).__init__(**kwargs)
    
    def compute_output_shape(self, input_shape):
        self._validate_input_shape(input_shape)

        if not self._return_attention:
            return input_shape[-1]
        else:
            return [input_shape[-1], [input_shape[0][0], input_shape[0][1], input_shape[1][2]]]
    
    def _validate_input_shape(self, input_shape):
        if len(input_shape) != 3:
            raise ValueError("Layer received an input shape {0} but expected three inputs (Q, V, K).".format(input_shape))
        else:
            if input_shape[0][0] != input_shape[1][0] or input_shape[1][0] != input_shape[2][0]:
                raise ValueError("All three inputs (Q, V, K) have to have the same batch size; received batch sizes: {0}, {1}, {2}".format(input_shape[0][0], input_shape[1][0], input_shape[2][0]))
            if input_shape[0][1] != input_shape[1][1] or input_shape[1][1] != input_shape[2][1]:
                raise ValueError("All three inputs (Q, V, K) have to have the same length; received lengths: {0}, {1}, {2}".format(input_shape[0][0], input_shape[1][0], input_shape[2][0]))
            if input_shape[0][2] != input_shape[1][2]:
                raise ValueError("Input shapes of Q {0} and V {1} do not match.".format(input_shape[0], input_shape[1]))
    
    def build(self, input_shape):
        self._validate_input_shape(input_shape)
        
        super(ScaledDotProductAttention, self).build(input_shape)
    
    def call(self, x, mask=None):
        q, k, v = x
        d_k = q.shape.as_list()[2]

        # in pure tensorflow:
        # weights = tf.matmul(x_batch, tf.transpose(y_batch, perm=[0, 2, 1]))
        # normalized_weights = tf.nn.softmax(weights/scaling)
        # output = tf.matmul(normalized_weights, x_batch)
        
        weights = K.batch_dot(q,  k, axes=[2, 2])

        if mask is not None:
            # add mask weights
            if isinstance(mask, (list, tuple)):
                if len(mask) > 0:
                    raise ValueError("mask can only be a Tensor or a list of length 1 containing a tensor.")

                mask = mask[0]

            weights += -1e10*(1-mask)

        normalized_weights = K.softmax(weights / np.sqrt(d_k))
        output = K.batch_dot(normalized_weights, v)
        
        if self._return_attention:
            return [output, normalized_weights]
        else:
            return output

    def get_config(self):
        config = {'return_attention': self._return_attention}
        base_config = super(ScaledDotProductAttention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

class MultiHeadAttention():
    """
        Implementation according to:
            "Attention is all you need" by A Vaswani, N Shazeer, N Parmar (2017)
    """

    def __init__(self, h, d_k=None, d_v=None, d_model=None, activation=None, return_attention=False, **kwargs):    
        super(MultiHeadAttention, self).__init__(**kwargs)
        
        if (type(h) is not int or h < 2):
            raise ValueError("You have to set `h` to an int >= 2.")
        self._h = h
        
        if d_model and (type(d_model) is not int or d_model < 1):
                raise ValueError("You have to set `d_model` to an int >= 1.")
        self._d_model = d_model
        
        if d_k and int (type(d_k) is not int or d_k < 1):
            raise ValueError("You have to set `d_k` to an int >= 1.")
        self._d_k = d_k
        
        if d_v and (type(d_v) is not int or d_v < 1):
            raise ValueError("You have to set `d_v` to an int >= 1.")
        self._d_v = d_v
        
        self._activation = None
        self._return_attention = return_attention
    
    def compute_output_shape(self, input_shape):
        self._validate_input_shape(input_shape)
        
        if self._return_attention:
            return [input_shape[-1], [input_shape[0][0], input_shape[1][1], self._h*input_shape[2][2]]]
        else:
            return input_shape[-1]
    
    def _validate_input_shape(self, input_shape):
        if len(input_shape) != 3:
            raise ValueError("Layer received an input shape {0} but expected three inputs (Q, V, K).".format(input_shape))
        else:
            if input_shape[0][0] != input_shape[1][0] or input_shape[1][0] != input_shape[2][0]:
                raise ValueError("All three inputs (Q, V, K) have to have the same batch size; received batch sizes: {0}, {1}, {2}".format(input_shape[0][0], input_shape[1][0], input_shape[2][0]))
            if input_shape[0][1] != input_shape[1][1] or input_shape[1][1] != input_shape[2][1]:
                raise ValueError("All three inputs (Q, V, K) have to have the same length; received lengths: {0}, {1}, {2}".format(input_shape[0][0], input_shape[1][0], input_shape[2][0]))
            if input_shape[0][2] != input_shape[1][2]:
                raise ValueError("Input shapes of Q {0} and V {1} do not match.".format(input_shape[0], input_shape[1]))
    
    def build(self, input_shape):
        self._validate_input_shape(input_shape)
        
        d_k = self._d_k if self._d_k else input_shape[1][-1]
        d_model = self._d_model if self._d_model else input_shape[1][-1]
        d_v = self._d_v

        if type(d_k) == tf.Dimension:
            d_k = d_k.value
        if type(d_model) == tf.Dimension:
            d_model = d_model.value
        
        self._q_layers = []
        self._k_layers = []
        self._v_layers = []
        self._sdp_layer = ScaledDotProductAttention(return_attention=self._return_attention)
    
        for _ in range(self._h):
            self._q_layers.append(
                TimeDistributed(
                    Dense(d_k, activation=self._activation, use_bias=False)
                )
            )
            self._k_layers.append(
                TimeDistributed(
                    Dense(d_k, activation=self._activation, use_bias=False)
                )
            )
            self._v_layers.append(
                TimeDistributed(
                    Dense(d_v, activation=self._activation, use_bias=False)
                )
            )
        
        self._output = TimeDistributed(Dense(d_model))
        #if self._return_attention:
        #    self._output = Concatenate()
    
    def __call__(self, x, mask=None):
        if isinstance(x, (list, tuple)):
            self.build([it.shape for it in x])
        else:
            self.build(x.shape)

        q, k, v = x
        
        outputs = []
        attentions = []
        for i in range(self._h):
            qi = self._q_layers[i](q)
            ki = self._k_layers[i](k)
            vi = self._v_layers[i](v)
            
            if self._return_attention:
                output, attention = self._sdp_layer([qi, ki, vi], mask=mask)
                outputs.append(output)
                attentions.append(attention)
            else:
                output = self._sdp_layer([qi, ki, vi], mask=mask)
                outputs.append(output)
            
        concatenated_outputs = Concatenate()(outputs)
        output = self._output(concatenated_outputs)
        
        if self._return_attention:
            attention = Concatenate()(attentions)
            # print("attention", attention, attention.shape)
       
        if self._return_attention:
            return [output, attention]
        else:
            return output        

# https://wanasit.github.io/attention-based-sequence-to-sequence-in-keras.html
# https://arxiv.org/pdf/1508.04025.pdf
class SequenceAttention(Layer):
    """
        Takes two inputs of the shape (batch_size, T, dim1) and (batch_size, T, dim2),
        whereby the first item is the source data and the second one the key data.
        This layer then calculates for each batch's element and each time step a softmax attention 
        vector between the key data and the source data. Finally, this attention vector is multiplied
        with the source data to obtain a weighted output. This means, that the key data is used to
        interpret the source data in a special way to create an output of the same shape as the source data.
    """
    def __init__(self, similarity, kernel_initializer="glorot_uniform", **kwargs):
        super(SequenceAttention, self).__init__(**kwargs)
        if isinstance(similarity, str):
            ALLOWED_SIMILARITIES = ["additive", "multiplicative" ]
            if similarity not in ALLOWED_SIMILARITIES:
                raise ValueError("`similarity` has to be either a callable or one of the following: {0}".format(ALLOWED_SIMILARITIES))
            else:
                self._similarity = getattr(self, "_" + similarity + "_similarity")
        elif callable(similarity):
            self._similarity = similarity
        else:
            raise ValueError("`similarity` has to be either a callable or one of the following: {0}".format(ALLOWED_SIMILARITIES))
            
        self._kernel_initializer = kernel_initializer
            
    def build(self, input_shape):
        super(SequenceAttention, self).build(input_shape)
        self._validate_input_shape(input_shape)
        
        self._weights = {}
        if self._similarity == self._additive_similarity:
            self._weights["w_a"] = self.add_weight(
                name='w_a', 
                shape=(input_shape[0][-1] + input_shape[1][-1], input_shape[0][-1]),
                initializer=self._kernel_initializer,
                trainable=True
            )
            
            self._weights["v_a"] = self.add_weight(
                name='v_a', 
                shape=(1, input_shape[0][-1]),
                initializer=self._kernel_initializer,
                trainable=True
            )
            
        elif self._similarity == self._multiplicative_similarity:
            self._weights["w_a"] = self.add_weight(
                name='w_a', 
                shape=(input_shape[1][-1], input_shape[0][-1]),
                initializer=self._kernel_initializer,
                trainable=True
            )

        self.built = True
        
    def compute_output_shape(self, input_shape):
        self._validate_input_shape(input_shape)
        
        return input_shape[0]
            
    def _validate_input_shape(self, input_shape):
        if len(input_shape) != 2:
            raise ValueError("Layer received an input shape {0} but expected two inputs (source, query).".format(input_shape))
        else:
            if input_shape[0][0] != input_shape[1][0]:
                raise ValueError("Both two inputs (source, query) have to have the same batch size; received batch sizes: {0}, {1}".format(input_shape[0][0], input_shape[1][0]))
            if input_shape[0][1] != input_shape[1][1]:
                raise ValueError("Both inputs (source, query) have to have the same length; received lengths: {0}, {1}".format(input_shape[0][0], input_shape[1][0]))
        
    def call(self, x):
        source, query = x
        
        similarity = self._similarity(source, query)
        expected_similarity_shape = [source.shape.as_list()[0], source.shape.as_list()[1], source.shape.as_list()[1]]
       
        if similarity.shape.as_list() != expected_similarity_shape:
            raise RuntimeError("The similarity function has returned a similarity with shape {0}, but expected {1}".format(similarity.shape.as_list()[:2], expected_similarity_shape))
        
        score = K.softmax(similarity)
        output = K.batch_dot(score, source, axes=[1, 1])
        
        return output
    
    def _additive_similarity(self, source, query):
        concatenation = K.concatenate([source, query], axis=2)
        nonlinearity = K.tanh(K.dot(concatenation, self._weights["w_a"]))
        
        # tile the weight vector (1, 1, dim) for each time step and each element of the batch -> (bs, T, dim)
        source_shape = K.shape(source)
        vaeff = K.tile(K.expand_dims(self._weights["v_a"], 0), [source_shape[0], source_shape[1], 1])

        similarity = K.batch_dot(K.permute_dimensions(vaeff, [0, 2, 1]), nonlinearity, axes=[1, 2])
        
        return similarity

    def _multiplicative_similarity(self, source, query):
        qp = K.dot(query, self._weights["w_a"])
        similarity = K.batch_dot(K.permute_dimensions(qp, [0, 2, 1]), source, axes=[1, 2])
        
        return similarity

    def get_config(self):
        config = {'similarity': self._similarity, 'kernel_initializer': self._kernel_initializer}
        base_config = super(SequenceAttention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

class AttentionRNNWrapper(Wrapper):
    """
        The idea of the implementation is based on the paper:
            "Effective Approaches to Attention-based Neural Machine Translation" by Luong et al.
        This layer is an attention layer, which can be wrapped around arbitrary RNN layers.
        This way, after each time step an attention vector is calculated
        based on the current output of the LSTM and the entire input time series.
        This attention vector is then used as a weight vector to choose special values
        from the input data. This data is then finally concatenated to the next input
        time step's data. On this a linear transformation in the same space as the input data's space
        is performed before the data is fed into the RNN cell again.
        This technique is similar to the input-feeding method described in the paper cited
    """

    def __init__(self, layer, weight_initializer="glorot_uniform", **kwargs):
        assert isinstance(layer, RNN)
        self.layer = layer
        self.supports_masking = True
        self.weight_initializer = weight_initializer
        
        super(AttentionRNNWrapper, self).__init__(layer, **kwargs)
        
    def _validate_input_shape(self, input_shape):
        if len(input_shape) != 3:
            raise ValueError("Layer received an input with shape {0} but expected a Tensor of rank 3.".format(input_shape[0]))

    def build(self, input_shape):
        self._validate_input_shape(input_shape)

        self.input_spec = InputSpec(shape=input_shape)
        
        if not self.layer.built:
            self.layer.build(input_shape)
            self.layer.built = True
            
        input_dim = input_shape[-1]

        if self.layer.return_sequences:
            output_dim = self.layer.compute_output_shape(input_shape)[0][-1]
        else:
            output_dim = self.layer.compute_output_shape(input_shape)[-1]
      
        self._W1 = self.add_weight(shape=(input_dim, input_dim), name="{}_W1".format(self.name), initializer=self.weight_initializer)
        self._W2 = self.add_weight(shape=(output_dim, input_dim), name="{}_W2".format(self.name), initializer=self.weight_initializer)
        self._W3 = self.add_weight(shape=(2*input_dim, input_dim), name="{}_W3".format(self.name), initializer=self.weight_initializer)
        self._b2 = self.add_weight(shape=(input_dim,), name="{}_b2".format(self.name), initializer=self.weight_initializer)
        self._b3 = self.add_weight(shape=(input_dim,), name="{}_b3".format(self.name), initializer=self.weight_initializer)
        self._V = self.add_weight(shape=(input_dim,1), name="{}_V".format(self.name), initializer=self.weight_initializer)
        
        super(AttentionRNNWrapper, self).build()
        
    def compute_output_shape(self, input_shape):
        self._validate_input_shape(input_shape)

        return self.layer.compute_output_shape(input_shape)
    
    @property
    def trainable_weights(self):
        return self._trainable_weights + self.layer.trainable_weights

    @property
    def non_trainable_weights(self):
        return self._non_trainable_weights + self.layer.non_trainable_weights

    def step(self, x, states):   
        h = states[0]
        # states[1] necessary?

        # equals K.dot(X, self._W1) + self._b2 with X.shape=[bs, T, input_dim]
        total_x_prod = states[-1]
        # comes from the constants (equals the input sequence)
        X = states[-2]
        
        # expand dims to add the vector which is only valid for this time step
        # to total_x_prod which is valid for all time steps
        hw = K.expand_dims(K.dot(h, self._W2), 1)
        additive_atn = total_x_prod + hw
        attention = K.softmax(K.dot(additive_atn, self._V), axis=1)
        x_weighted = K.sum(attention * X, [1])

        x = K.dot(K.concatenate([x, x_weighted], 1), self._W3) + self._b3
        
        h, new_states = self.layer.cell.call(x, states[:-2])
        
        return h, new_states
    
    def call(self, x, constants=None, mask=None, initial_state=None):
        # input shape: (n_samples, time (padded with zeros), input_dim)
        input_shape = self.input_spec.shape

        if self.layer.stateful:
            initial_states = self.layer.states
        elif initial_state is not None:
            initial_states = initial_state
            if not isinstance(initial_states, (list, tuple)):
                initial_states = [initial_states]

            base_initial_state = self.layer.get_initial_state(x)
            if len(base_initial_state) != len(initial_states):
                raise ValueError("initial_state does not have the correct length. Received length {0} but expected {1}".format(len(initial_states), len(base_initial_state)))
            else:
                # check the state' shape
                for i in range(len(initial_states)):
                    if not initial_states[i].shape.is_compatible_with(base_initial_state[i].shape): #initial_states[i][j] != base_initial_state[i][j]:
                        raise ValueError("initial_state does not match the default base state of the layer. Received {0} but expected {1}".format([x.shape for x in initial_states], [x.shape for x in base_initial_state]))
        else:
            initial_states = self.layer.get_initial_state(x)
            
        if not constants:
            constants = []
            
        constants += self.get_constants(x)
        
        last_output, outputs, states = K.rnn(
            self.step,
            x,
            initial_states,
            go_backwards=self.layer.go_backwards,
            mask=mask,
            constants=constants,
            unroll=self.layer.unroll,
            input_length=input_shape[1]
        )
        
        if self.layer.stateful:
            self.updates = []
            for i in range(len(states)):
                self.updates.append((self.layer.states[i], states[i]))

        if self.layer.return_sequences:
            output = outputs
        else:
            output = last_output 

        # Properly set learning phase
        if getattr(last_output, '_uses_learning_phase', False):
            output._uses_learning_phase = True
            for state in states:
                state._uses_learning_phase = True

        if self.layer.return_state:
            if not isinstance(states, (list, tuple)):
                states = [states]
            else:
                states = list(states)
            return [output] + states
        else:
            return output

    def get_constants(self, x):
        # add constants to speed up calculation
        constants = [x, K.dot(x, self._W1) + self._b2]
        
        return constants

    def get_config(self):
        config = {'weight_initializer': self.weight_initializer}
        base_config = super(AttentionRNNWrapper, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

class ExternalAttentionRNNWrapper(Wrapper):
    """
        The basic idea of the implementation is based on the paper:
            "Effective Approaches to Attention-based Neural Machine Translation" by Luong et al.
        This layer is an attention layer, which can be wrapped around arbitrary RNN layers.
        This way, after each time step an attention vector is calculated
        based on the current output of the LSTM and the entire input time series.
        This attention vector is then used as a weight vector to choose special values
        from the input data. This data is then finally concatenated to the next input
        time step's data. On this a linear transformation in the same space as the input data's space
        is performed before the data is fed into the RNN cell again.
        This technique is similar to the input-feeding method described in the paper cited.
        The only difference compared to the AttentionRNNWrapper is, that this layer
        applies the attention layer not on the time-depending input but on a second
        time-independent input (like image clues) as described in:
            Show, Attend and Tell: Neural Image Caption Generation with Visual Attention
            https://arxiv.org/abs/1502.03044
    """
    def __init__(self, layer, weight_initializer="glorot_uniform", return_attention=False, **kwargs):
        assert isinstance(layer, RNN)
        self.layer = layer
        self.supports_masking = True
        self.weight_initializer = weight_initializer
        self.return_attention = return_attention
        self._num_constants = None

        super(ExternalAttentionRNNWrapper, self).__init__(layer, **kwargs)

        self.input_spec = [InputSpec(ndim=3), InputSpec(ndim=3)]
        
    def _validate_input_shape(self, input_shape):
        if len(input_shape) >= 2:
            if len(input_shape[:2]) != 2:
                raise ValueError("Layer has to receive two inputs: the temporal signal and the external signal which is constant for all time steps")
            if len(input_shape[0]) != 3:
                raise ValueError("Layer received a temporal input with shape {0} but expected a Tensor of rank 3.".format(input_shape[0]))
            if len(input_shape[1]) != 3:
                raise ValueError("Layer received a time-independent input with shape {0} but expected a Tensor of rank 3.".format(input_shape[1]))
        else:
            raise ValueError("Layer has to receive at least 2 inputs: the temporal signal and the external signal which is constant for all time steps")

    def build(self, input_shape):
        self._validate_input_shape(input_shape)

        for i, x in enumerate(input_shape):
            self.input_spec[i] = InputSpec(shape=x)
        
        if not self.layer.built:
            self.layer.build(input_shape)
            self.layer.built = True
            
        temporal_input_dim = input_shape[0][-1]
        static_input_dim = input_shape[1][-1]

        if self.layer.return_sequences:
            output_dim = self.layer.compute_output_shape(input_shape[0])[0][-1]
        else:
            output_dim = self.layer.compute_output_shape(input_shape[0])[-1]
      
        self._W1 = self.add_weight(shape=(static_input_dim, temporal_input_dim), name="{}_W1".format(self.name), initializer=self.weight_initializer)
        self._W2 = self.add_weight(shape=(output_dim, temporal_input_dim), name="{}_W2".format(self.name), initializer=self.weight_initializer)
        self._W3 = self.add_weight(shape=(temporal_input_dim + static_input_dim, temporal_input_dim), name="{}_W3".format(self.name), initializer=self.weight_initializer)
        self._b2 = self.add_weight(shape=(temporal_input_dim,), name="{}_b2".format(self.name), initializer=self.weight_initializer)
        self._b3 = self.add_weight(shape=(temporal_input_dim,), name="{}_b3".format(self.name), initializer=self.weight_initializer)
        self._V = self.add_weight(shape=(temporal_input_dim, 1), name="{}_V".format(self.name), initializer=self.weight_initializer)
        
        super(ExternalAttentionRNNWrapper, self).build()
        
    @property
    def trainable_weights(self):
        return self._trainable_weights + self.layer.trainable_weights

    @property
    def non_trainable_weights(self):
        return self._non_trainable_weights + self.layer.non_trainable_weights

    def compute_output_shape(self, input_shape):
        self._validate_input_shape(input_shape)

        output_shape =  self.layer.compute_output_shape(input_shape[0])

        if self.return_attention:
            if not isinstance(output_shape, list):
                output_shape = [output_shape]

            output_shape = output_shape + [(None, input_shape[1][1])]

        return output_shape
    
    def step(self, x, states):  
        h = states[0]
        # states[1] necessary?
        
        # comes from the constants
        X_static = states[-2]
        # equals K.dot(static_x, self._W1) + self._b2 with X.shape=[bs, L, static_input_dim]
        total_x_static_prod = states[-1]

        # expand dims to add the vector which is only valid for this time step
        # to total_x_prod which is valid for all time steps
        hw = K.expand_dims(K.dot(h, self._W2), 1)
        additive_atn = total_x_static_prod + hw
        attention = K.softmax(K.dot(additive_atn, self._V), axis=1)
        static_x_weighted = K.sum(attention * X_static, [1])
        
        x = K.dot(K.concatenate([x, static_x_weighted], 1), self._W3) + self._b3

        h, new_states = self.layer.cell.call(x, states[:-2])
        
        # append attention to the states to "smuggle" it out of the RNN wrapper
        attention = K.squeeze(attention, -1)
        h = K.concatenate([h, attention])

        return h, new_states
    
    def call(self, x, constants=None, mask=None, initial_state=None):
        # input shape: (n_samples, time (padded with zeros), input_dim)
        input_shape = self.input_spec[0].shape

        if len(x) > 2:
            initial_state = x[2:]
            x = x[:2]
            assert len(initial_state) >= 1

        static_x = x[1]
        x = x[0]

        if self.layer.stateful:
            initial_states = self.layer.states
        elif initial_state is not None:
            initial_states = initial_state
            if not isinstance(initial_states, (list, tuple)):
                initial_states = [initial_states]
        else:
            initial_states = self.layer.get_initial_state(x)
            
        if not constants:
            constants = []
        constants += self.get_constants(static_x)

        last_output, outputs, states = K.rnn(
            self.step,
            x,
            initial_states,
            go_backwards=self.layer.go_backwards,
            mask=mask,
            constants=constants,
            unroll=self.layer.unroll,
            input_length=input_shape[1]
        )

        # output has at the moment the form:
        # (real_output, attention)
        # split this now up

        output_dim = self.layer.compute_output_shape(input_shape)[0][-1]
        last_output = last_output[:output_dim]

        attentions = outputs[:, :, output_dim:]
        outputs = outputs[:, :, :output_dim]
        
        if self.layer.stateful:
            self.updates = []
            for i in range(len(states)):
                self.updates.append((self.layer.states[i], states[i]))

        if self.layer.return_sequences:
            output = outputs
        else:
            output = last_output 

        # Properly set learning phase
        if getattr(last_output, '_uses_learning_phase', False):
            output._uses_learning_phase = True
            for state in states:
                state._uses_learning_phase = True

        if self.layer.return_state:
            if not isinstance(states, (list, tuple)):
                states = [states]
            else:
                states = list(states)
            output = [output] + states

        if self.return_attention:
            if not isinstance(output, list):
                output = [output]
            output = output + [attentions]

        return output

    def _standardize_args(self, inputs, initial_state, constants, num_constants):
        """Standardize `__call__` to a single list of tensor inputs.
        When running a model loaded from file, the input tensors
        `initial_state` and `constants` can be passed to `RNN.__call__` as part
        of `inputs` instead of by the dedicated keyword arguments. This method
        makes sure the arguments are separated and that `initial_state` and
        `constants` are lists of tensors (or None).
        # Arguments
        inputs: tensor or list/tuple of tensors
        initial_state: tensor or list of tensors or None
        constants: tensor or list of tensors or None
        # Returns
        inputs: tensor
        initial_state: list of tensors or None
        constants: list of tensors or None
        """
        inputs=inputs[:2]
        if isinstance(inputs, list) and len(inputs) > 2:
            assert initial_state is None and constants is None
            if num_constants is not None:
                constants = inputs[-num_constants:]
                inputs = inputs[:-num_constants]
            initial_state = inputs[2:]
            inputs = inputs[:2]

        def to_list_or_none(x):
            if x is None or isinstance(x, list):
                return x
            if isinstance(x, tuple):
                return list(x)
            return [x]

        initial_state = to_list_or_none(initial_state)
        constants = to_list_or_none(constants)

        return inputs, initial_state, constants

    def __call__(self, inputs, initial_state=None, constants=None, **kwargs):
        inputs, initial_state, constants = self._standardize_args(
            inputs, initial_state, constants, self._num_constants)

        if initial_state is None and constants is None:
            return super(ExternalAttentionRNNWrapper, self).__call__(inputs, **kwargs)

        # If any of `initial_state` or `constants` are specified and are Keras
        # tensors, then add them to the inputs and temporarily modify the
        # input_spec to include them.

        additional_inputs = []
        additional_specs = []
        if initial_state is not None:
            kwargs['initial_state'] = initial_state
            additional_inputs += initial_state
            self.state_spec = [InputSpec(shape=K.int_shape(state))
                               for state in initial_state]
            additional_specs += self.state_spec
        if constants is not None:
            kwargs['constants'] = constants
            additional_inputs += constants
            self.constants_spec = [InputSpec(shape=K.int_shape(constant))
                                   for constant in constants]
            self._num_constants = len(constants)
            additional_specs += self.constants_spec
        # at this point additional_inputs cannot be empty
        is_keras_tensor = K.is_keras_tensor(additional_inputs[0])
        for tensor in additional_inputs:
            if K.is_keras_tensor(tensor) != is_keras_tensor:
                raise ValueError('The initial state or constants of an ExternalAttentionRNNWrapper'
                                 ' layer cannot be specified with a mix of'
                                 ' Keras tensors and non-Keras tensors'
                                 ' (a "Keras tensor" is a tensor that was'
                                 ' returned by a Keras layer, or by `Input`)')

        if is_keras_tensor:
            # Compute the full input spec, including state and constants
            full_input = inputs + additional_inputs
            full_input_spec = self.input_spec + additional_specs
            # Perform the call with temporarily replaced input_spec
            original_input_spec = self.input_spec
            self.input_spec = full_input_spec
            output = super(ExternalAttentionRNNWrapper, self).__call__(full_input, **kwargs)
            self.input_spec = self.input_spec[:len(original_input_spec)]
            return output
        else:
            return super(ExternalAttentionRNNWrapper, self).__call__(inputs, **kwargs)

    def get_constants(self, x):
        # add constants to speed up calculation
        constants = [x, K.dot(x, self._W1) + self._b2]
        return constants

    def get_config(self):
        config = {'return_attention': self.return_attention, 'weight_initializer': self.weight_initializer}
        base_config = super(ExternalAttentionRNNWrapper, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
#from kulc.attention import ExternalAttentionRNNWrapper
from tensorflow.keras.layers import Input,Embedding,Lambda,Dense,TimeDistributed,LSTM,Reshape
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from tensorflow.keras.utils import plot_model

def create_model(vocabulary_size, embedding_size, T, L, D):
    
    image_input = Input(shape=(256,256,3), name="image_input")
    
    image_model = tf.keras.applications.VGG16(include_top=False,
                                          input_shape=(256,256,3),
                                          weights='imagenet')
    image_model = Model(image_model.input, image_model.layers[-2].output)
    for layer in image_model.layers:
        layer.trainable = False
    
    
    image_features_input = image_model(image_input)
    image_features_input = Reshape((16*16,512))(image_features_input)

    captions_input = Input(shape=(T,), name="captions_input")
    captions = Embedding(vocabulary_size, embedding_size, input_length=T)(captions_input)

    averaged_image_features = Lambda(lambda x: K.mean(x, axis=1))
    averaged_image_features = averaged_image_features(image_features_input)
    initial_state_h = Dense(embedding_size)(averaged_image_features)
    initial_state_c = Dense(embedding_size)(averaged_image_features)

    image_features = TimeDistributed(Dense(D, activation="relu" ))(image_features_input)

    encoder = LSTM(embedding_size, return_sequences=True, return_state=True, recurrent_dropout=0.1)
    attented_encoder = ExternalAttentionRNNWrapper(encoder, return_attention=True)

    output = TimeDistributed(Dense(vocabulary_size, activation="softmax"), name="output")

    # for training purpose
    attented_encoder_training_data, _, _ , _= attented_encoder([captions, image_features], initial_state=[initial_state_h, initial_state_c])
    
    
    training_output_data = output(attented_encoder_training_data)

    training_model = Model(inputs=[image_input,captions_input], outputs=training_output_data)
    
    initial_state_inference_model = Model(inputs=[image_input], outputs=[initial_state_h, initial_state_c])
    
    inference_initial_state_h = Input(shape=(embedding_size,))
    inference_initial_state_c = Input(shape=(embedding_size,))
    attented_encoder_inference_data, inference_encoder_state_h, inference_encoder_state_c, inference_attention = attented_encoder(
        [captions, image_features],
        initial_state=[inference_initial_state_h, inference_initial_state_c]
        )
   
    inference_output_data = output(attented_encoder_inference_data)

    inference_model = Model(
        inputs=[image_input, captions_input, inference_initial_state_h, inference_initial_state_c],
        outputs=[inference_output_data, inference_encoder_state_h, inference_encoder_state_c, inference_attention]
    )
    
    return training_model, inference_model, initial_state_inference_model

In [ ]:
embedding_size = 512
T= None
L= 16*16
D= 512
training_model, inference_model, initial_state_inference_model = create_model(vocab_size, embedding_size, T, L, D)

In [ ]:
plot_model(training_model)

In [ ]:
training_model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam
epochs = 5
lr=1e-3
training_model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(lr))

In [ ]:
hist = training_model.fit_generator( train_dataloader,
                    validation_data = val_dataloader,
                    epochs = epochs
                )

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

def tokens_to_text(tokens,tok,end_token='endseq'):
    sentence=""
    for token in tokens:
        if token ==0:
            break
        
        word = tok.index_word[token]
        
        if word==end_token:
            break
            
        sentence+= word+" "
        
    sentence = sentence.strip()
    
    return sentence


def greedy_inference(input_img, tok,encoder_model, decoder_model,max_len,start_token="startseq",end_token='endseq',decoder_type="LSTM"):
    if decoder_type=='LSTM':
        a0,c0  =encoder_model(np.expand_dims(input_img,axis=0))
    elif decoder_type=='GRU': 
        hidden_layer  =encoder_model(np.expand_dims(input_img,axis=0))
        
    word = tok.word_index[start_token]
    
    words = []
    
    for index in range(max_len):
        if decoder_type=='LSTM':
            word_probs , a0,c0 = decoder_model.predict([[np.array([word]),a0,c0]])
        elif decoder_type=='GRU': 
            word_probs , hidden_layer = decoder_model.predict([[np.array([word]),hidden_layer]])
            hidden_layer=hidden_layer[0]
        
        word = np.argmax(word_probs)
        
        if tok.index_word[word]==end_token:
            break
        
        words.append(word)
        
    words = tokens_to_text(words,tok)
    return words

In [ ]:
def get_predictions_from_data_loader(data_loader,tok,encoder_model, decoder_model,max_len,start_token="startseq",end_token='endseq', inference_type='greedy',decoder_type='LSTM'):
    
    data_loader_iterator = data_loader.__iter__()
    
    pred_sentences = []
    Gt_sentences = []
    for index, (X,Y) in enumerate(data_loader_iterator):
        for img,_,sample_y in zip(X[0],X[1],Y):
            
            if inference_type=='greedy':
                pred_sentence = greedy_inference(img, tok,encoder_model, decoder_model,max_len,start_token="startseq",end_token='endseq',decoder_type='LSTM')
            
            GT_sentence   = tokens_to_text(sample_y,tok)
            
            pred_sentences.append(pred_sentence)
            Gt_sentences.append(GT_sentence)
        
        if index == data_loader.nb_iteration -1:
            break
        print(index)
        
    return Gt_sentences, pred_sentences

In [ ]:
def calculate_bleu_evaluation(GT_sentences, predicted_sentences):
    BLEU_1 = corpus_bleu(GT_sentences, predicted_sentences, weights=(1.0, 0, 0, 0))
    BLEU_2 = corpus_bleu(GT_sentences, predicted_sentences, weights=(0.5, 0.5, 0, 0))
    BLEU_3 = corpus_bleu(GT_sentences, predicted_sentences, weights=(0.3, 0.3, 0.3, 0))
    BLEU_4 = corpus_bleu(GT_sentences, predicted_sentences, weights=(0.25, 0.25, 0.25, 0.25))
    
    return BLEU_1,BLEU_2,BLEU_3,BLEU_4

In [ ]:
def evaluate_from_dataloader(data_loader,tok,encoder_model, decoder_model,max_len,start_token="startseq",end_token='endseq', inference_type='greedy',decoder_type="LSTM"):
    Gt_sentences, pred_sentences = get_predictions_from_data_loader(data_loader,tok,encoder_model, decoder_model,max_len,start_token=start_token,end_token=end_token, inference_type=inference_type,decoder_type=decoder_type)
    BLEU_1,BLEU_2,BLEU_3,BLEU_4 = calculate_bleu_evaluation(Gt_sentences, pred_sentences)
    
    return BLEU_1,BLEU_2,BLEU_3,BLEU_4

In [ ]:
evaluate_from_dataloader(val_dataloader,tok,encoder_model, decoder_model,max_len,decoder_type="LSTM")

In [ ]:
# Seq 2 Seq model

(0.1388382130461867, 0.3726100012696743, 0.553035111006621, 0.6104178906861055)






# Dr's Model

In [ ]:
layers.LSTM?

In [ ]:
define_model(vocab_size, max_len)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam


from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
from MultiCheXNet.data_loader.indiana_dataloader import get_train_validation_generator

max_vocab_size=10000
max_len=100

csv_path1  ="/kaggle/input/chest-xrays-indiana-university/indiana_reports.csv"
csv_path2  ="/kaggle/input/chest-xrays-indiana-university/indiana_projections.csv"
img_path   ="/kaggle/input/chest-xrays-indiana-university/images/images_normalized/"

train_dataloader, val_dataloader, vocab_size, tok = get_train_validation_generator(csv_path1,csv_path2,img_path, max_vocab_size,max_len,dim=(224,224), preprocess=preprocess_input)

In [ ]:
X,Y= next(enumerate(train_dataloader))[1]


In [ ]:
# define the captioning model
def define_model(vocab_size, max_length):
    
    ## encoder
    inputs1 = layers.Input(shape=(224,224,3))
    VGG16 = tf.keras.applications.VGG16(
            include_top=True,
            weights="imagenet",
            input_shape=(224,224,3),
        )
    
    VGG16_output = Model(inputs=VGG16.inputs, outputs=VGG16.layers[-2].output)(inputs1)
    fe1 = layers.Dropout(0.5)(VGG16_output)
    fe2 = layers.Dense(256, activation='relu')(fe1)
    
    
    #decoder layers
    embd_layer =  layers.Embedding(vocab_size, 256, mask_zero=True)
    dropout_layer = layers.Dropout(0.5)
    lstm_layer = layers.LSTM(256)
    dense_1 = layers.Dense(256, activation='relu')
    dense_2 = layers.Dense(vocab_size, activation='softmax')
    
    # sequence model
    inputs2 = layers.Input(shape=(max_length,))
    se1 = embd_layer(inputs2)
    se2 = dropout_layer(se1)
    se3 = lstm_layer(se2)
    # decoder model
    decoder1 = layers.add([fe2, se3])
    decoder2 = dense_1(decoder1)
    outputs = dense_2(decoder2)
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    # summarize model
    
    
    encoder_model = Model(inputs=inputs1, outputs=fe2)
    
    
    #decoder_model 
    img_features = layers.Input(shape=(256,))
    inputs2 = layers.Input(shape=(1,))
    
    se1 = embd_layer(inputs2)
    se2 = dropout_layer(se1)
    se3 = lstm_layer(se2)
    
    decoder1 = layers.add([img_features, se3])
    decoder2 = dense_1(decoder1)
    outputs = dense_2(decoder2) 
    
    decoder_model = Model(inputs=[inputs2, img_features], outputs=outputs)
    
    print(model.summary())
    plot_model(model, to_file='model.png', show_shapes=True)
    return model,encoder_model, decoder_model

In [ ]:
model,encoder_model, decoder_model = define_model(vocab_size, max_len)

In [ ]:
decoder_model.summary()

In [ ]:
Y.shape

In [ ]:
lr=1e-3
epochs = 5

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr))

In [ ]:
Y.shape

In [ ]:
hist = model.fit_generator( 
                    train_dataloader,
                    epochs = epochs
                )